In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Flatten
import pickle

In [2]:
IoT = pd.read_csv('../../data/IoT_Motion_Light.csv')
IoT = IoT.iloc[0:500]

IoT = IoT.dropna()

encoder=LabelEncoder()
IoT['type']=encoder.fit_transform(IoT['type'])
IoT['light_status']=encoder.fit_transform(IoT['light_status'])

In [3]:
x = IoT.drop(['label', 'date', 'time'], axis=1)
y = IoT['label']

features=len(x.columns)

x_train,x_test,y_train,y_test = train_test_split(x, y, test_size=0.2)

In [4]:
model1 = Sequential()
    
model1.add(LSTM(units = 128, activation='tanh', return_sequences=True, input_shape = (features,1)))
model1.add(Dropout(0.2))
model1.add(LSTM(units = 64, activation='tanh', return_sequences=True))
model1.add(Dropout(0.2))
model1.add(Dense(1, activation='sigmoid')) 
model1.add(Flatten())
    
model1.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [5]:
model1.save('../h5s/LSTM.h5') 
# filename = '../h5s/LSTM.h5'
# pickle.dump(model1, open(filename, 'wb'))

In [6]:
model1.fit(x_train, np.asarray(y_train).astype('float32').reshape((-1,1)), epochs = 5, batch_size=32, validation_data=(x_test, np.asarray(y_test).astype('float32').reshape((-1,1))))

Epoch 1/5
13/13 [==============================] - 8s 145ms/step - loss: 0.5775 - accuracy: 0.9800 - val_loss: 0.4154 - val_accuracy: 1.0000
Epoch 2/5
13/13 [==============================] - 0s 17ms/step - loss: 0.3136 - accuracy: 1.0000 - val_loss: 0.2319 - val_accuracy: 1.0000
Epoch 3/5
13/13 [==============================] - 0s 15ms/step - loss: 0.1924 - accuracy: 1.0000 - val_loss: 0.1577 - val_accuracy: 1.0000
Epoch 4/5
13/13 [==============================] - 0s 16ms/step - loss: 0.1324 - accuracy: 1.0000 - val_loss: 0.1094 - val_accuracy: 1.0000
Epoch 5/5
13/13 [==============================] - 0s 18ms/step - loss: 0.0922 - accuracy: 1.0000 - val_loss: 0.0764 - val_accuracy: 1.0000


In [8]:
predictions = model1.predict(x_test)
y_pred = [round(x[0]) for x in predictions]
y_pred

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [9]:
for i in range(3):
	print(x_test.values[i].tolist(), '=>', y_pred[i], '( expected ', y_train.values[i], ')')

[0, 0, 0] => 0 ( expected  0 )
[0, 0, 0] => 0 ( expected  0 )
[1, 1, 0] => 0 ( expected  0 )


In [10]:
score, acc = model1.evaluate(x_test, y_test, batch_size=1)

print('Test score:', score)
print('Test accuracy:', acc)

100/100 [==============================] - 3s 4ms/step - loss: 0.0764 - accuracy: 1.0000
Test score: 0.07640799134969711
Test accuracy: 1.0
